In [1]:
# Connect it all up

In [124]:
import json
from pathlib import Path

def get_current_variables():
    try:
        return json.loads(Path('data.json').read_text())
    except FileNotFoundError:
        print('could not load JSON so returning empty')
        return {
            'not_found': [],
            'firstpage_urls': {},
            'newspaper_urls': {},
            'page_urls': {},
            'ambiguous': []
        }

def get_current_data():
    try:
        json_data = json.loads(Path('billboard-data.json').read_text())
        return {k: v for k, v in json_data.items() if not k == '_investigate'}
    except FileNotFoundError:
        return {
            '_investigate': []
        }


In [82]:
def get_latest_sources():
    import pandas as pd
    import re

    ID_NUMBERS = re.compile(r'\d{6,10}')
    DATES = re.compile(r'(January|February|March|April|May|June|July|August|September|October|November|December) (\d{1,2}), (\d{4})')
    ENDS_WITH_PAGE = re.compile(r', (\d{1,2})$')

    df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=254069133&single=true&output=csv')
    sources = [x for x in df['Source clean']]
    sources = sorted([x for x in sources if not ID_NUMBERS.search(x) and DATES.search(x) and ENDS_WITH_PAGE.search(x)])
    return sources

In [164]:
page_urls = get_current_variables()['page_urls']
billboard = get_current_data()

for line in get_latest_sources():
    line = str(line)
    if DATES.search(line) and ENDS_WITH_PAGE.search(line):
        found = None
        paper = DATES.split(line)[0][:-2]
        paper = re.sub(' [A-Z][A-Z] ', ' ', paper)
        date = DATES.search(line).groups()
        page = ENDS_WITH_PAGE.search(line).groups()[0]
        if paper.lower() == 'billboard':
            found = billboard.get(date[2], {}).get(date[0], {}).get(date[1], {}).get(page, None)
            
            if found:
                found = [x for x in found.split('/') if x and re.search('^[0-9]*$', x)][0]
        else:
            if page_urls.get(paper):
                date_as_string = f'{date[2]}-{date[0]}-{date[1]}'

                if date_as_string in page_urls[paper]:
                    if page_urls[paper][date_as_string].get(page):
                        found = [x for x in page_urls.get(paper).get(date_as_string).get(page).split('/') if x][-1]

        if found:
            print(line, '\t', found)

Cincinnati Enquirer, December 30, 1937, 6 	 99598407
Cincinnati Enquirer, December 30, 1937, 8 	 103410794
